In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='4'
import time
import numpy as np
import pandas as pd
import tensorflow as tf
from VGG16_GAP import VGG16_GAP

In [ ]:
import pickle
import imgaug as ia
from imgaug import augmenters as iaa

In [ ]:
import numpy as np
import skimage.io as imageio

def readImgList(file_list):
    images = list()
    for i, file in enumerate(file_list):
        print(i, end="\r")
        img = imageio.imread(file)
        img = img.astype(int)
        images.append(img)
    return np.array(images)

def transformLabel(id_list, y_dict):
    label = list()
    for uid in list(id_list):
        label.append(y_dict[uid])
    return np.array(label)

def one_hot_encoding(class_numbers, num_classes):
    return np.eye(num_classes, dtype=float)[class_numbers]

In [ ]:
with open('save/label_dict.pkl', 'rb') as f:
    y_dict = pickle.load(f)

In [ ]:
HOME_DIR = "/home/cmchang/DLCV2018SPRING/final/"
TRAIN_DIR = HOME_DIR+"dlcv_final_2_dataset/train/"
VALID_DIR = HOME_DIR+"dlcv_final_2_dataset/val/"

In [ ]:
dtrain = pd.read_csv(HOME_DIR+"dlcv_final_2_dataset/train_id.txt", header=None,sep=" ", names=["img", "id"])
dvalid = pd.read_csv(HOME_DIR+"dlcv_final_2_dataset/val_id.txt", header=None,sep=" ", names=["img", "id"])

In [ ]:
train_list = list(TRAIN_DIR+dtrain.img)
valid_list = list(VALID_DIR+dvalid.img)

In [ ]:
def initialize_uninitialized(sess):
    global_vars = tf.global_variables()
    is_not_initialized = sess.run([tf.is_variable_initialized(var) for var in global_vars])
    not_initialized_vars = [v for (v,f) in zip(global_vars, is_not_initialized) if not f]
    if len(not_initialized_vars): 
            sess.run(tf.variables_initializer(not_initialized_vars))

In [ ]:
Xtrain = readImgList(train_list)

In [ ]:
Xvalid = readImgList(valid_list)

In [ ]:
Xtrain.shape

In [ ]:
ytrain = transformLabel(list(dtrain.id), y_dict)

In [ ]:
yvalid = transformLabel(list(dvalid.id), y_dict)

In [ ]:
Ytrain = one_hot_encoding(ytrain, len(y_dict))
Yvalid = one_hot_encoding(yvalid, len(y_dict))

In [ ]:
scope_name = "Model22"

In [ ]:
model = VGG16_GAP(scope_name=scope_name)

In [ ]:
SELF_DIR = "newCL_v5_lambda-1e-1_dynamic_gap_L5_v3_rescale0-1_save_linear/"
FLAG_init_from = HOME_DIR+SELF_DIR+"para_dict.npy"
FLAG_prof_type = "linear"
FLAG_lambda_s = 4e-3
FLAG_lambda_m = 2e-5
FLAG_decay = 1e-5
FLAG_lr = 2e-6
FLAG_keep_prob = 1.0
FLAG_lambda_c = 1e-3
FLAG_save_dir = HOME_DIR+"jus_test/"

In [ ]:
model.build(vgg16_npy_path=FLAG_init_from,
            shape=Xtrain.shape[1:],
            classes=len(y_dict),
            prof_type=FLAG_prof_type,
            conv_pre_training=True,
            fc_pre_training=True,
            new_bn=False)

In [ ]:
centers = np.load(HOME_DIR+SELF_DIR+"centers.npy")
model.add_centers(centers.astype(np.float32))

In [ ]:
dp = [1.0, 0.75, 0.5]
tasks = [str(int(p*100)) for p in dp]
model.set_idp_operation(dp=dp, decay=FLAG_decay, keep_prob=FLAG_keep_prob, lambda_c = FLAG_lambda_c)

In [ ]:
obj = 0.0
for cur_task in tasks:
    print(cur_task)
    obj += model.loss_dict[cur_task]

In [ ]:
tracking = list()
for cur_task in tasks:
    tracking.append(model.accu_dict[cur_task])

In [ ]:
sometimes = lambda aug: iaa.Sometimes(0.5, aug)
transform = iaa.Sequential([
    sometimes(iaa.Affine(translate_percent={"x": (-0.15, 0.15), "y": (-0.15, 0.15)})),
    sometimes(iaa.Affine(scale={"x": (0.85, 1.15), "y":(0.85, 1.15)})),
    sometimes(iaa.Affine(rotate=(-45, 45))),
    sometimes(iaa.Fliplr(0.5))
])

In [ ]:
print("===== create directory =====")
if not os.path.exists(FLAG_save_dir):
    os.makedirs(FLAG_save_dir)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    augment = True

   # hyper parameters
    batch_size = 32
    epoch = 100
    early_stop_patience = 10
    min_delta = 0.0001

    # recorder
    epoch_counter = 0
    history = list()

    # Passing global_step to minimize() will increment it at each step.
    learning_rate = FLAG_lr
    opt = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.5)

    checkpoint_path = os.path.join(FLAG_save_dir, 'model.ckpt')
    
    # trainable variables
    train_vars = list()
    for var in tf.trainable_variables():
        if model.scope_name in var.name:
            train_vars.append(var)
    
    for var in train_vars:
        if '_mean' in var.name:
            train_vars.remove(var)
            print('%s is not trainable.'% var)
    
    for var in train_vars:
        if '_variance' in var.name:
            train_vars.remove(var)
            print('%s is not trainable.'% var)
    
    print(train_vars)
            
    train_op = opt.minimize(obj, var_list=train_vars)
    
    saver = tf.train.Saver(tf.global_variables(), max_to_keep=len(tasks))

    # max step in a epoch
    ptrain_max = int(Xtrain.shape[0]/batch_size)
    pval_max = int(Xvalid.shape[0]/batch_size)

    # re-initialize
    initialize_uninitialized(sess)

    # reset due to adding a new task
    patience_counter = 0
    current_best_val_accu = 0

    # optimize when the aggregated obj
    while(patience_counter < early_stop_patience and epoch_counter < epoch):
        
        # start training
        stime = time.time()
        train_loss, train_accu = 0.0, 0.0
        
        if augment:
            def load_batches():
                for i in range(int(Xtrain.shape[0]/batch_size)):
                    print("Training: {0}/{1}".format(i,ptrain_max), end='\r')
                    st = i*batch_size
                    ed = (i+1)*batch_size
                    batch = ia.Batch(images=Xtrain[st:ed,:,:,:], data=Ytrain[st:ed,:])
                    yield batch

            batch_loader = ia.BatchLoader(load_batches)
            bg_augmenter = ia.BackgroundAugmenter(batch_loader=batch_loader, augseq=transform, nb_workers=1)

            while True:
                batch = bg_augmenter.get_batch()
                if batch is None:
                    print("Finished epoch.")
                    break
                x_images_aug = batch.images_aug
                y_images = batch.data
                loss, accu, _, _ = sess.run([obj, model.accu_dict[cur_task], train_op, model.centers_update_op], 
                                         feed_dict={model.x: x_images_aug,
                                                    model.y: y_images,
                                                    model.is_train: True,
                                                    model.bn_train: False})
                train_loss += loss
                train_accu += accu
            batch_loader.terminate()
            bg_augmenter.terminate()
        else:
            for i in range(int(Xtrain.shape[0]/batch_size)):
                print("Training: {0}/{1}".format(i,ptrain_max), end='\r')
                st = i*batch_size
                ed = (i+1)*batch_size
                loss, accu, _, _ = sess.run([obj, model.accu_dict[tasks[0]], train_op, model.centers_update_op],
                                                    feed_dict={model.x: Xtrain[st:ed,:],
                                                               model.y: Ytrain[st:ed,:],
                                                               model.is_train: True,
                                                               model.bn_train: False})
                train_loss += loss
                train_accu += accu

        train_loss = train_loss/ptrain_max
        train_accu = train_accu/ptrain_max


        # validation
        val_loss, val_accu1, val_accu2 = 0.0, 0.0, 0.0
        val_accu_dp = list()
        for i in range(int(Xvalid.shape[0]/batch_size)):
            print("Validating: {0}/{1}".format(i,pval_max), end='\r')
            st = i*batch_size
            ed = (i+1)*batch_size
            loss, accu1, accu2, accu_dp = sess.run([obj, model.accu_dict[tasks[0]], model.accu_dict[tasks[-1]], tracking],
                                                feed_dict={model.x: Xvalid[st:ed,:],
                                                           model.y: Yvalid[st:ed,:],
                                                           model.is_train: False,
                                                           model.bn_train: False})
            val_loss += loss
            val_accu1 += accu1
            val_accu2 += accu2
            val_accu_dp.append(accu_dp)
            
        val_accu_dp = np.mean(val_accu_dp, axis=0).tolist()
        dp_str = ""
        for i in range(len(tasks)):
            dp_str += "{0}%:{1}, ".format(tasks[i], np.round(val_accu_dp[i],4))
        
        print(dp_str)
        val_loss = val_loss/pval_max
        val_accu1 = val_accu1/pval_max
        val_accu2 = val_accu2/pval_max
        val_accu = val_accu1 # used for early stopping
        
        # early stopping check
        if (val_accu - current_best_val_accu) > min_delta:
            current_best_val_accu = val_accu
            patience_counter = 0

            para_dict = sess.run(model.para_dict)
            np.save(os.path.join(FLAG_save_dir, "para_dict.npy"), para_dict)
            print("save in %s" % os.path.join(FLAG_save_dir, "para_dict.npy"))
        else:
            patience_counter += 1

        # shuffle Xtrain and Ytrain in the next epoch
        idx = np.random.permutation(Xtrain.shape[0])
        Xtrain, Ytrain = Xtrain[idx,:,:,:], Ytrain[idx,:]

        # epoch end
        epoch_counter += 1

        print("Epoch %s (%s), %s sec >> train loss: %.4f, train accu: %.4f, val loss: %.4f, val accu at %s: %.4f, val accu at %s: %.4f" % (epoch_counter, patience_counter, round(time.time()-stime,2), train_loss, train_accu, val_loss, tasks[0], val_accu1, tasks[-1], val_accu2))
        history.append([train_loss, train_accu, val_loss, val_accu ])
        
        if epoch_counter % 10 == 0:
            import matplotlib.pyplot as plt
            df = pd.DataFrame(history)
            df.columns = ['train_loss', 'train_accu', 'val_loss', 'val_accu']
            df[['train_loss', 'val_loss']].plot()
            plt.savefig(os.path.join(FLAG_save_dir, 'loss.png'))
            plt.close()
            df[['train_accu', 'val_accu']].plot()
            plt.savefig(os.path.join(FLAG_save_dir, 'accu.png'))
            plt.close()
            
    saver.save(sess, checkpoint_path, global_step=epoch_counter)
    
    # extract features and calculate center

    output = []
    for i in range(int(Xtrain.shape[0]/200+1)):
        print(i, end="\r")
        st = i*200
        ed = min((i+1)*200, Xtrain.shape[0])
        prob = sess.run(model.features, feed_dict={model.x: Xtrain[st:ed,:], 
                                                   model.is_train: False,
                                                   model.bn_train: False})
        output.append(prob)

    for i in range(int(Xvalid.shape[0]/200+1)):
        print(i, end="\r")
        st = i*200
        ed = min((i+1)*200, Xvalid.shape[0])
        prob = sess.run(model.features, feed_dict={model.x: Xvalid[st:ed,:], 
                                                   model.is_train: False,
                                                   model.bn_train: False})
        output.append(prob)

    EX = np.concatenate(output)
    print(EX.shape)
    EY = np.concatenate([ytrain, yvalid])
    print(EY.shape)
    centers = np.zeros((len(y_dict), EX.shape[1]))
    for i in range(len(y_dict)):
        centers[i,:] = np.mean(EX[EY==i,:], axis=0)
        np.save(arr=centers,file=os.path.join(FLAG_save_dir,"centers.npy"))